**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.9MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N8b1f8dd8444e42469e52f225de56227c (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person


In [11]:
from rdflib.plugins.sparql import prepareQuery

q = prepareQuery('''
  SELECT ?s
  WHERE {
    ?s rdfs:subClassOf ns:Person.
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns}
)

for r in g.query(q):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [13]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s) 

for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for x, y, z in g.triples((None, RDF.type, s)):
    print(x)

http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


In [17]:
q = prepareQuery('''
  SELECT ?s
  WHERE {
    {
      ?s rdf:type ns:Person.
    } UNION {
      ?sub rdfs:subClassOf ns:Person.
      ?s rdf:type ?sub
    }
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns, "rdf":RDF}
)

for r in g.query(q):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [18]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for x, y, z in g.triples((s, None, None)):
      print(x, y, z)

for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for x, y, z in g.triples((None, RDF.type, s)):
    for a, b, c in g.triples((x, None, None)):
      print(a, b, c)

http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/

In [21]:
q = prepareQuery('''
  SELECT ?s ?p ?c
  WHERE {
    {
      ?s rdf:type ns:Person.
      ?s ?p ?c
    } UNION {
      ?s rdf:type ?sub.
      ?s ?p ?c.
      ?sub rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = {"rdfs":RDFS, "ns":ns, "rdf":RDF}
)

for r in g.query(q):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/